内置类型（使用 C 语言编写）不会调用用户定义的类覆盖的特殊方法
- [Subclasses of built-in types](https://doc.pypy.org/en/latest/cpython_differences.html#subclasses-of-built-in-types)

原生类型的行为违背了面向对象编程的一个基本原则：始终应该从实例（self）所属的类开始搜索方法，即使在超类实现的类中调用也是如此。

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

# 超类 __init__ 方法覆盖子类 __setitem__ 方法
dd = DoppelDict(one=1)
dd

{'one': 1}

In [2]:
# [] 调用子类 __setitem__  方法
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [3]:
# 超类方法
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

内置类型的方法调用其他类的方法，如果被覆盖了，也不会被调用

不要子类化内置类型，用户自己定义的类应该继承 collections 模块

上述问题只发生在 C 语言实现的内置类型内部的方法委托上，而且只影响直接继承内置类型的用户自定义类。

In [4]:
class AnswerDict(dict):
    def __getitem__(self, key):  # 始终返回 42
        return 42

ad = AnswerDict(a='foo')
ad['a']

42

In [5]:
d = {}
d.update(ad)  # 忽略了 AnswerDict 中的方法
d['a']

'foo'

In [6]:
d

{'a': 'foo'}

In [7]:
# 子类化 collections.UserDict
import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [8]:
dd['two'] = 2
dd

{'one': [1, 1], 'two': [2, 2]}

In [9]:
dd.update(three=3)
dd

{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}

In [10]:
# 子类化 collections.UserDict
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict2(a='foo')
ad

{'a': 'foo'}

In [11]:
d = {}
d.update(ad)
d['a']

42

In [12]:
d

{'a': 42}

菱形问题：由不相关的祖先类实现同名方法引起的命名冲突。

Python 按照方法解析顺序（Method Resolution Order, MRO）遍历继承图。类都有一个名为 \_\_mro__ 的属性，它的值是一个元组，按照方法解析顺序列出各个超类，从当前类一直向上，直到 object 类。

In [13]:
class A:
    def ping(self):
        print('ping:', self)


class B(A):
    def pong(self):
        print('pong:', self)


class C(A):
    def pong(self):
        print('PONG:', self)


class D(B, C):

    def ping(self):
        super().ping()
        print('post-ping:', self)

    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [14]:
d = D()
d.pong()  # B

pong: <__main__.D object at 0x000001DD7759AC10>


In [15]:
# 超类中的方法可以直接调用，把实例作为显示参数传入
C.pong(d)

PONG: <__main__.D object at 0x000001DD7759AC10>


In [16]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

若想把方法委托给超类，推荐的方式是使用内置的 super() 函数。使用 super() 最安全，也不易过时，使用 super() 调用方法时，会遵守方法解析顺序

直接在类上调用实例方法时，必须显式传入 self 参数，因为这样访问的是未绑定方法（unbound method）

方法解析顺序不仅考虑继承图，还考虑子类声明中列出超类的顺序。方法解析顺序使用 C3 算法计算。

In [17]:
# 两次调用
d.ping()

ping: <__main__.D object at 0x000001DD7759AC10>
post-ping: <__main__.D object at 0x000001DD7759AC10>


In [18]:
# self.ping()     D 类
# 
# super.ping()    A 类
# self.pong()     B 类
# super().pong()  B 类
# C.pong(self)    C 类

d.pingpong()

ping: <__main__.D object at 0x000001DD7759AC10>
post-ping: <__main__.D object at 0x000001DD7759AC10>
ping: <__main__.D object at 0x000001DD7759AC10>
pong: <__main__.D object at 0x000001DD7759AC10>
pong: <__main__.D object at 0x000001DD7759AC10>
PONG: <__main__.D object at 0x000001DD7759AC10>


一些常用类的方法搜索顺序

In [19]:
bool.__mro__

(bool, int, object)

In [20]:
import numbers

# numbers 模块提供的几个数字抽象基类
numbers.Integral.__mro__

(numbers.Integral,
 numbers.Rational,
 numbers.Real,
 numbers.Complex,
 numbers.Number,
 object)

In [21]:
# io 模块中有抽象基类（名称以 ...Base 后缀结尾）和具体类
import io

io.BytesIO.__mro__

(_io.BytesIO, _io._BufferedIOBase, _io._IOBase, object)

In [22]:
io.TextIOWrapper.__mro__

(_io.TextIOWrapper, _io._TextIOBase, _io._IOBase, object)

处理多重继承

1. 把接口继承和实现继承区分开

    使用多重继承时，一定要明确一开始为什么创建子类。
    - 继承接口，创建子类型，实现“是什么”关系
    - 继承实现，通过重用避免代码重复


2. 使用抽象基类显式表示接口

    如果类的作用是定义接口，应该明确把它定义为抽象基类。


3. 通过混入重用代码

    如果一个类的作用是为多个不相关的子类提供方法实现，从而实现重用，但不体现“是什么”关系，应该把那个类明确地定义为混入类（mixin class）
    
    从概念上讲，混入不定义新类型，只是打包方法，便于重用。混入类绝对不能实例化，而且具体类不能只继承混入类。


4. 在名称中明确指明混入

    在 Python 中没有把类声明为混入的正规方式，所以推荐在名称中加入 ...Mixin 后缀。


5. 抽象基类可以作为混入，反过来不成立

    抽象基类可以实现具体方法，因此也可以作为混入使用
    - 抽象基类会定义类型，而混入做不到
    - 抽象基类可以作为其他类的唯一基类，而混入决不能作为唯一的超类
    
    抽象基类中实现的具体方法只能与抽象基类及其超类中的方法协作。


6. 不要子类化多个具体类

    具体类可以没有。或最多只有一个具体超类。具体类的超类中出了这一个具体超类之外，其余的都是抽象基类或混入。


7. 为用户提供聚合类

    如果一个类的结构主要继承自混入，自身没有添加结构或行为，那么这样的类称为聚合类。


8. “优先使用对象组合，而不是类继承”

    优先使用组合能让设计更灵活。子类化是一种紧耦合，而且较高的继承树容易倒。
    
    组合和委托可以代替混入，把行为提供给不同的类，但是不能取代接口继承去定义类型层次结构。

Django 的分派机制是动态版模板方法模式。

基于类的视图能避免大量样板代码，便于重用，还能增进团队交流。

如果作为应用程序开发者，你发现自己在构建多层类层次结构，可能是发生了下述事件中的一个或多个：
- 你在重新发明轮子。去找框架或库，它们提供的组件可以在应用程序中重用。
- 你使用的框架设计不良。去寻找替代品。
- 你在过度设计。记住要遵守 KISS 原则。
- 你厌烦了编写应用程序，决定新造一个框架。